In [221]:
import os
import spacy
import numpy as np
#
import pandas as pd
#
from spacy.matcher import Matcher

In [222]:
nlp = spacy.load('de_core_news_md')
pd.set_option('display.max_colwidth', None)
os.environ["SPACY_WARNING_IGNORE"] = "W008"

In [223]:
REVIEWS_CLEANED_FILE_PATH = 'data/reviews_merged_unlabeled_cleaned.csv'

df = pd.read_csv(REVIEWS_CLEANED_FILE_PATH, sep=';')
df.head()

,Unnamed: 0,caption,rating
0,0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch,4.0
1,1,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter,5.0
2,2,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult,5.0
3,3,gestern dritt selben burger bestellt gestern abend magen darm burger lecker irgendetwas schmeckte komisch erwartet,1.0
4,4,fanden burger restaurant klasse individuelle möglichkeiten zudem fleischlose alternativen ambiente zeitsprung vergangenheit kartenzahlung heutzutage zeitgemäß,4.0


In [224]:
categories = ['essen', 'gaststättengewerbe', 'atmosphäre', 'preis', 'warten']

In [225]:
def spacy_most_similar(word, topn=10):
    ms = nlp.vocab.vectors.most_similar(
        nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    distances = ms[2]
    return words, distances

In [226]:
spacy_most_similar(categories[0])

(['Aufessen',
  'weggefuttert',
  'drinken',
  'einzukochen',
  'koten',
  'inhaliere',
  'knabbere',
  'Bio-Essen',
  'schmatz',
  'Trinket'],
 array([[1.    , 0.8243, 0.8131, 0.7734, 0.7115, 0.7059, 0.7032, 0.6845,
         0.672 , 0.6681]], dtype=float32))

In [300]:
matcher = Matcher(nlp.vocab)
patterns = [
    [{'POS':'ADJ', 'OP': '+'}, {'POS':'NOUN'}],
    [{'POS':'NOUN'}, {'POS':'VERB'}, {'POS':'ADJ'}]
]
matcher.add("category", patterns)

df_matches = pd.DataFrame([], columns=['index', 'matches'])

similarity_min = 0.4

for rowIndex, wordText in enumerate(df['caption'][:5]):
    doc = nlp(wordText)
    matches = matcher(doc)
    #print(rowIndex, len(matches))

    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        #print(span)
        for i, s in enumerate(span):
            if s.pos_ in ['NOUN']:
                similarity_cat0 = nlp(categories[0]).similarity(span)
                if similarity_cat0 > similarity_min: # fits into
                    print(rowIndex, ' - ', categories[0], ' - ', span, ' - ', similarity_cat0)

                similarity_cat1 = nlp(categories[1]).similarity(span)
                if similarity_cat1 > similarity_min: # fits into
                    print(rowIndex, ' - ', categories[1], ' - ', span, ' - ', similarity_cat1)

                similarity_cat2 = nlp(categories[2]).similarity(span)
                if similarity_cat2 > similarity_min: # fits into
                    print(rowIndex, ' - ', categories[2], ' - ', span, ' - ', similarity_cat2)

                similarity_cat3 = nlp(categories[3]).similarity(span)
                if similarity_cat3 > similarity_min: # fits into
                    print(rowIndex, ' - ', categories[3], ' - ', span, ' - ', similarity_cat3)

                similarity_cat4 = nlp(categories[4]).similarity(span)
                if similarity_cat4 > similarity_min: # fits into
                    print(rowIndex, ' - ', categories[4], ' - ', span, ' - ', similarity_cat4)
    df_matches = pd.concat([pd.DataFrame({'index': rowIndex, 'matches': matches}, columns=df_matches.columns), df_matches])

#print(len(df_matches))

/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/2378623975.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/2378623975.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/2378623975.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)


0  -  atmosphäre  -  ambiente konzept  -  0.568398898436853
0  -  atmosphäre  -  cooles ambiente konzept  -  0.5555121262381959
0  -  essen  -  lecker bürger  -  0.4209613832566812
1  -  atmosphäre  -  sympathische mitarbeiter  -  0.41114811017893294
1  -  atmosphäre  -  gemütliche atmosphäre  -  0.9041401865935277
1  -  atmosphäre  -  freundliche gemütliche atmosphäre  -  0.8451887596877069
1  -  essen  -  fantastisches essen  -  0.9070816153259378
4  -  atmosphäre  -  individuelle möglichkeiten  -  0.42664221280523584
4  -  atmosphäre  -  ambiente zeitsprung  -  0.7064088538909967
4  -  atmosphäre  -  alternativen ambiente zeitsprung  -  0.6628601520202437
4  -  atmosphäre  -  fleischlose alternativen ambiente zeitsprung  -  0.5270339242481952


/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/2378623975.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/2378623975.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)


In [228]:
similarity_min = 0.4

for rowIndex, wordText in enumerate(df['caption']):
    word = nlp(wordText)
    tokens = [token.text for token in word]
    #tokens
    allowed_postags = ["NOUN"]
    for i, token in enumerate(word):
        if token.pos_ in allowed_postags:
            if token and token.vector_norm > 1:
                similarity_cat0 = nlp(categories[0]).similarity(token)
                if similarity_cat0 > similarity_min: # fits into
                    print(rowIndex, categories[0], token, similarity_cat0)

                similarity_cat1 = nlp(categories[1]).similarity(token)
                if similarity_cat1 > similarity_min: # fits into
                    print(rowIndex, categories[1], token, similarity_cat1)

                similarity_cat2 = nlp(categories[2]).similarity(token)
                if similarity_cat2 > similarity_min: # fits into
                   print(rowIndex, categories[2], token, similarity_cat2)

                similarity_cat3 = nlp(categories[3]).similarity(token)
                if similarity_cat3 > similarity_min: # fits into
                    print(rowIndex, categories[3], token, similarity_cat3)

                similarity_cat4 = nlp(categories[4]).similarity(token)
                if similarity_cat2 > similarity_min: # fits into
                   print(rowIndex, categories[4], token, similarity_cat4)


1 atmosphäre atmosphäre 1.0
1 warten atmosphäre 0.15941722217303403
1 essen essen 1.0


/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/1235100666.py:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(token)


In [229]:
nlp("essen").similarity(nlp("pizza"))

0.4386588382204547

In [230]:
nlp('pizz').vector_norm

0